In [0]:
#Necessary Imports
!pip install gensim
import gensim
import warnings
warnings.filterwarnings("ignore")
import numpy as np



#**Data Processing**

###**Opening File**

In [3]:
import os 
!ls
os.chdir('/content/drive/My Drive/Colab Notebooks/Word Emphasis Selection/')
f = open("train.txt", "r")

drive  sample_data


#### **Reading File Line By Line**

In [0]:
lines=f.readlines()

#### **Spliting Lines by tab and saving values in lists**

In [0]:
sent=[[] for i in lines]                    ### initializing list
probs=[[] for i in lines]                   ### Intializing list 

j=0
word=''
prob=''
for i in range(0,len(lines)):               ### loop on each line of file
  cols=lines[i].strip().split('\t')         ### spliting line on the basis of tab
  if (len(cols)>2):                         ### if line has data
    sent[j].append(cols[1])                 ### Appending word in list named sent
    probs[j].append(cols[4])                ### Appending probs in list named probs
  else:
    j=j+1

#### **Removing empty sublists from list**

In [0]:
sent = [x for x in sent if x != []]

In [0]:
probs = [x for x in probs if x != []]

#### **Displaying List conatining tokenized words of all the sentences**

In [16]:
print (sent)

[['Save', 'the', 'Date'], ['The', 'habit', 'of', 'persistence', 'is', 'the', 'habit', 'of', 'victory', '.'], ['What', "'s", 'dangerous', 'is', 'not', 'to', 'evolve', '.'], ['UN', 'REAL'], ['If', 'you', 'have', 'the', 'ability', 'to', 'love', ',', 'love', 'yourself', 'first', '.'], ['I', 'would', 'rather', 'entertain', 'and', 'hope', 'that', 'people', 'learned', 'something', 'than', 'educate', 'people', 'and', 'hope', 'they', 'were', 'entertained', '.'], ['Happy', 'Friendship', 'Day', '!'], ['Genius', 'is', 'one', 'percent', 'inspiration', 'and', 'ninety-nine', 'percent', 'perspiration', '.'], ['Join', 'the', '#plantgang', 'and', 'add', 'to', 'your', 'jungalow', '!'], ['If', 'you', 'want', 'to', 'improve', ',', 'be', 'content', 'to', 'be', 'thought', 'foolish', 'and', 'stupid', '.'], ['Orange', 'you', 'glad', 'you', 'came', 'to', 'the', 'party', '?'], ['READING', 'RULES', '.'], ['When', 'you', 'want', 'to', 'succeed', 'as', 'bad', 'as', 'you', 'want', 'to', 'breathe', ',', 'then', 'you'

#### **Displaying list containing probabilities of words**

In [17]:
print (probs)

[['0.8888888888888888', '0.4444444444444444', '0.7777777777777778'], ['0.0', '0.3333333333333333', '0.0', '0.6666666666666666', '0.0', '0.0', '0.1111111111111111', '0.0', '0.7777777777777778', '0.1111111111111111'], ['0.0', '0.0', '0.5555555555555556', '0.0', '0.2222222222222222', '0.2222222222222222', '0.5555555555555556', '0.1111111111111111'], ['0.6666666666666666', '0.8888888888888888'], ['0.0', '0.0', '0.0', '0.0', '0.2222222222222222', '0.1111111111111111', '0.3333333333333333', '0.0', '0.7777777777777778', '0.7777777777777778', '0.6666666666666666', '0.2222222222222222'], ['0.0', '0.0', '0.1111111111111111', '0.7777777777777778', '0.1111111111111111', '0.2222222222222222', '0.0', '0.2222222222222222', '0.8888888888888888', '0.2222222222222222', '0.0', '0.5555555555555556', '0.1111111111111111', '0.1111111111111111', '0.2222222222222222', '0.1111111111111111', '0.1111111111111111', '0.3333333333333333', '0.1111111111111111'], ['0.2222222222222222', '1.0', '0.4444444444444444', '0

##**Training Word2Vec to learn embeddings**

In [0]:
max_features = 150
max_words = 38
# instantiating Word2Vec to train the model by passing the list of lists
# build vocabulary and train model
model = gensim.models.Word2Vec(
    sent,
    size=max_features,
    window=10,
    min_count=1,
    workers=10,
    iter=30)

**Function to find embeddings**

In [0]:
'''  *******************************************************************
       Returns Combined Embeddings of Title and Description of an Issue
     *******************************************************************  '''

def get_sentence_embeddings(sentence_tokens):
  
    #to store embeddings of all the words in the sentence    
    sent_embeddings= np.zeros([max_words,max_features ])
    i = 0
    #iterating over all the words in the title one by one
    for word in sentence_tokens:
        
        sent_embeddings[i] = np.array(model[word])  #embedding of a single word
        i=i+1
        
    if len(sentence_tokens) < max_words:
      for i in range(len(sentence_tokens), max_words):
        sent_embeddings[i] = np.zeros(max_features)
        i = i +1
         
    #mean embedding of all the words in title      
    #mean_embedings = np.mean(np.array(sent_embeddings),axis=0)  
    
    return np.array(sent_embeddings)

In [0]:
data={}
X = []
Y= []
for i in range(len(sent)):
    
    sentence_embeddings = get_sentence_embeddings(sent[i])
    #print ('Sent[0]',sentence_embeddings[0])
    tmp = []
    for prob in probs[i]:
      tmp.append(float(prob))  
    if len(tmp) < 38:
      for j in range(len(tmp), 38):
        tmp.append(float(0))
    X.append(sentence_embeddings)
    Y.append(np.array(tmp))
   
    data[i]=[]  #initializing empty list at current dictionary index 
   
    #Storing Sentences, their Embeddings and probabilities of all the words in sentence 
    data[i].append(sent[i])
    data[i].append(sentence_embeddings)
    data[i].append(probs[i])
    
    
    
    

In [0]:
X = np.array(X)
Y = np.array(Y)

In [55]:
X.shape

(2742, 38, 150)

##Declaring our Model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding,Bidirectional
from keras.layers import LSTM

max_features = 150
SEQUENCE_LEN = max_words
lstm_model = Sequential()
lstm_model.add(Bidirectional(LSTM(128), input_shape=(SEQUENCE_LEN, max_features)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(max_words, activation='sigmoid'))

lstm_model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])


###Loading Saved weights

In [0]:
#Loading saved weights

from keras.models import load_model
 
# load model
lstm_model.load_model('model.h5')

###Training Part

In [85]:

lstm_model.fit(X[:1650,:,:],Y[:1650,:], batch_size=10, epochs=100)
#score = model.evaluate(x_test, y_test, batch_size=16)


Epoch 1/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0166 - acc: 0.3224
Epoch 2/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0162 - acc: 0.3061
Epoch 3/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0160 - acc: 0.3152
Epoch 4/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0159 - acc: 0.3176
Epoch 5/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0156 - acc: 0.3376
Epoch 6/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0154 - acc: 0.3248
Epoch 7/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0151 - acc: 0.3164
Epoch 8/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0150 - acc: 0.3333
Epoch 9/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0148 - acc: 0.3370
Epoch 10/100
1650/1650 [==============================] - 20s 12ms/step - loss: 0.0146 - acc: 0.3430

###Results on Testset

In [87]:
lstm_model.evaluate(X[1650:,:,:],Y[1650:,:], batch_size=16)

1092/1092 [==============================] - 4s 3ms/step


[0.0154714889571944, 0.3232600732600733]

####Results on a single example

In [101]:
predicted_probs = lstm_model.predict(X[0:1,:,:])[0]  #0.5 0.4 0.8 
number_of_words_in_given_sentence = len(data[0][0])
print('Predicted Probs: ',predicted_probs[:number_of_words_in_given_sentence])
print('Original Probs: ',Y[0,:number_of_words_in_given_sentence])

Predicted Probs:  [0.55178225 0.4019184  0.83689326]
Original Probs:  [0.88888889 0.44444444 0.77777778]


###Saving Weights

In [102]:
# saving weights to HDF5
	
!sudo pip install h5py
lstm_model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
